# 1) Import libraries and define global variables

In [25]:
import numpy as np
from tqdm import tqdm
import itertools
import plotly.graph_objects as go
import plotly.express as px



WORKERS_NB = 20 #number of workers
K_MIN = 10 #Minimum routes that can be created to pick up trash. K_MAX is actually WORKERS_NB

STREETS_NB = 100 #number of streets in the city with potential trash to collect
CLEAN_MAX = 70 #maximum streets that needs to be cleaned each week
CLEAN_MIN = 50 #minimum streets that needs to be cleaned each week

NB_WEEKS_SIM = 15 #number of weeks to simulate for the records

NB_WEEKS_AFTER = 15 #number of weeks to simulate after the records (for part 5)

GAMMA = 0.9 #discount factor -> how much we value the past

# 2) Functions to generate streets to clean, possible routes and matches with workers

In [26]:
def generate_streets():
    """
    Generate a random matrix of format STREETS_NB x 2 that contains the coordinates of the streets. The coordinates are floats between 0 and 100
    """
    city_map = np.random.rand(STREETS_NB, 2)*100

    #For each streets, calculate the distance to all other streets
    distances = np.zeros((STREETS_NB, STREETS_NB))

    for i in range(STREETS_NB):
        for j in range(STREETS_NB):
            if i != j:
                distances[i, j] = np.linalg.norm(city_map[i] - city_map[j])


    return city_map, distances



def generate_streets_to_clean():
    """
    Generate a random array of 0s and 1s of size STREETS_NB, where 1s represent the streets that need to be cleaned
    """
    n = np.random.randint(CLEAN_MIN, CLEAN_MAX + 1) #number of streets to clean

    streets = np.zeros(STREETS_NB)
    streets[:n] = 1 #Set the first n streets to 1, meaning they need to be cleaned
    np.random.shuffle(streets) #Shuffle the array to randomize the streets that need to be cleaned

    return streets


def generate_routes(streets, distances) :
    """
    Generate the routes for the workers to clean the streets. The routes are generated using a probabilistic approach, where the distance between the streets is used as a weight to assign the streets to the workers
    """

    workers_needed = np.random.randint(K_MIN, WORKERS_NB + 1) #number of workers needed to clean the streets
    routes = [[] for _ in range(workers_needed)] #List of routes for each worker

    #Get the index of the streets that need to be cleaned
    streets_to_clean = np.where(streets == 1)[0]
    np.random.shuffle(streets_to_clean)


    for i in range(len(streets_to_clean)):

        if i < workers_needed : #First, assign a street to each route so that each route has at least one street to clean
            routes[i].append(streets_to_clean[i])

        else : #For each route, take the distance to the closest routes and use it as a proba weight to assign the street to the route
            
            distance_vector = distances[streets_to_clean[i], :]
            proba_weight = [np.inf]*workers_needed

            for j in range(workers_needed):
                
                for k in routes[j]:
                    if distance_vector[k] < proba_weight[j]:
                        proba_weight[j] = distance_vector[k]

            
            proba_weight = np.array(proba_weight)
            #Normalize between 0 and 1
            proba_weight = proba_weight / np.sum(proba_weight)

            #Probabilistic assignment
            route_index = np.random.choice(range(workers_needed), p=proba_weight)
            routes[route_index].append(streets_to_clean[i])

    #Reorder the routes : use the Travelling salesman problem heuristic : nearest neighbor
    new_routes = []
    distances_travel = []
    for route in routes:
        new_route = []
        distance_traveled = 0
        for i in range(len(route)):
            if i == 0:
                new_route.append(route[0])
                route = np.delete(route, 0)
            else:
                distance_vector = distances[new_route[-1], route]
                next_street = np.argmin(distance_vector)
                new_route.append(route[next_street])
                route = np.delete(route, next_street)

                distance_traveled += distance_vector[next_street]

        new_routes.append(new_route)
        distances_travel.append(distance_traveled)

    return new_routes, distances_travel


def generate_match_workers(routes) :
    """
    """
    workers_match = [-1]*WORKERS_NB #-1 means that the worker is not matched with a route (they don't drive, stay at maintenance)
    workers_match[:len(routes)] = range(len(routes)) #The first workers are matched with a route
    np.random.shuffle(workers_match) #Shuffle the array to randomize the workers that are matched with a route

    routes_match = [0]*len(routes) #The number of workers matched with each route (reverse function of workers_match)
    for k in range(len(workers_match)):
        if workers_match[k] != -1:
            routes_match[workers_match[k]] += k
    
    return np.array(workers_match), np.array(routes_match )
        

street_map, distances = generate_streets()
streets_to_clean = generate_streets_to_clean()
generated_routes, distances_travel = generate_routes(streets_to_clean, distances)
workers_match, routes_match = generate_match_workers(generated_routes)


#### TEST PRINT ######
print("TEST generated routes <--> workers matched :\n")
for i, route in enumerate(generated_routes) : 
    print(f"{route} (distance of {round(distances_travel[i],2)}) <--> {routes_match[i]}")

#Get the index of workers_match that are equals to -1
maintenance_workers = np.where(np.array(workers_match) == -1)[0]
print(f"\nMaintenance workers : {maintenance_workers}")



TEST generated routes <--> workers matched :

[87, 9, 25, 92] (distance of 136.5) <--> 7
[37, 2, 91] (distance of 86.49) <--> 13
[26, 13] (distance of 48.4) <--> 3
[51, 16] (distance of 48.71) <--> 18
[38, 30, 73] (distance of 100.27) <--> 11
[85, 63, 29, 78] (distance of 183.43) <--> 19
[21, 65] (distance of 78.96) <--> 12
[75, 89] (distance of 58.56) <--> 2
[66, 39, 56, 3] (distance of 161.05) <--> 10
[82, 46, 64] (distance of 102.93) <--> 6
[96, 10, 88] (distance of 100.21) <--> 16
[15, 84, 99, 53, 74] (distance of 156.29) <--> 1
[32, 67, 36, 61, 58, 7, 28, 20] (distance of 235.75) <--> 0
[40] (distance of 0) <--> 15
[6, 59, 18] (distance of 101.37) <--> 8
[77, 97] (distance of 72.57) <--> 5
[22, 72, 8] (distance of 92.62) <--> 4

Maintenance workers : [ 9 14 17]


# 3) Creation of route-assignment records

### Create the city

In [27]:
street_map, distances = generate_streets()

### Simulate `NB_WEEKS_SIM` weeks

In [28]:
simulated_weeks = [] #List of weeks simulated [{streets_to_clean, generated_routes, distances_travel, workers_match, routes_match}, ...]
#streets_to_clean = [0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0] -> 1 when the street needs to be cleaned
#generated_routes = [[1, 2, 4, 6, 8, 10, 12, 14, 16, 18], [0, 3, 5, 7, 9, 11, 13, 15, 17, 19]] -> list of routes (route are sequence of streets to clean)
#distances_travel = [12.3, 15.2] -> distance to travel for each route of generated_routes
#workers_match = [1, -1, 0, -1, -1] -> index of the route that the worker is matched with. -1 means that the worker is not matched with a route (they don't drive, stay at maintenance)
#routes_match = [2, 0] -> index of workers matched with each route (reverse function of workers_match)


for i in tqdm(range(NB_WEEKS_SIM), desc="Simulating weeks..."):

    week_info = {}

    streets_to_clean = generate_streets_to_clean()
    generated_routes, distances_travel = generate_routes(streets_to_clean, distances)
    workers_match, routes_match = generate_match_workers(generated_routes)

    week_info["streets_to_clean"] = streets_to_clean
    week_info["generated_routes"] = generated_routes
    week_info["distances_travel"] = distances_travel
    week_info["workers_match"] = workers_match
    week_info["routes_match"] = routes_match

    simulated_weeks.append(week_info)


def retrieve_info(simulated_weeks):
    """
    Calculate information about the simulation of each worker
    """
    info = {}
    sim_week = len(simulated_weeks)
    for i in tqdm(range(sim_week), desc="Retrieving information..."):
        for w in range(WORKERS_NB):
            route_of_worker = simulated_weeks[i]["workers_match"][w]
            if route_of_worker != -1: #Not in maintenance
                
                #INFORMATION TO RETRIEVE
                driving_index = GAMMA**(sim_week - i)
                total_distance = simulated_weeks[i]["distances_travel"][route_of_worker]

                if w not in info: #Initialize the info
                    info[w] = {"driving_index": 0, "total_distance": 0, "visited_streets": {}}

                info[w]["driving_index"] += driving_index
                info[w]["total_distance"] += total_distance
                if i >= sim_week - 3 : #If the week is in the last 3 weeks, store the visited streets
                    info[w]["visited_streets"][sim_week - i] =  simulated_weeks[i]["generated_routes"][route_of_worker]

    return info

info = retrieve_info(simulated_weeks)
print("\n\nINFO :")
for w in info:
    print(f"Worker {w} : {info[w]}")

Retrieving information...: 100%|██████████| 15/15 [00:00<00:00, 117159.33it/s]



INFO :
Worker 0 : {'driving_index': 5.23294131505816, 'total_distance': 1546.8764676050564, 'visited_streets': {3: [79, 92, 94, 67, 97, 36], 2: [36, 13, 56, 69, 88, 10, 28], 1: [55, 53, 31, 9]}}
Worker 1 : {'driving_index': 6.4771297856671595, 'total_distance': 1831.2618277594127, 'visited_streets': {3: [18, 44, 81, 47, 7, 3], 2: [74, 39, 57, 98, 17, 21], 1: [83, 15, 75, 14]}}
Worker 2 : {'driving_index': 5.55810391001755, 'total_distance': 1509.375544132276, 'visited_streets': {2: [30, 68, 49, 91], 1: [43, 81]}}
Worker 5 : {'driving_index': 5.44422418759855, 'total_distance': 1536.1364953336622, 'visited_streets': {3: [98, 40, 6, 56, 57, 68, 49, 62], 2: [97, 58, 72, 46, 1, 90], 1: [68, 39, 86]}}
Worker 6 : {'driving_index': 5.70159958183426, 'total_distance': 1688.2989798788763, 'visited_streets': {3: [59, 75, 90, 89, 51, 32, 31], 2: [34, 12, 94, 84, 89, 70, 40], 1: [29, 91]}}
Worker 7 : {'driving_index': 5.46353356495816, 'total_distance': 2054.642656811898, 'visited_streets': {3: 

# 4) Apply our trees for next week

### Get next week's routes

In [29]:
streets_to_clean = generate_streets_to_clean()
generated_routes, distances_travel = generate_routes(streets_to_clean, distances)

maintenance_number = WORKERS_NB - len(generated_routes)


#Put the info in a list of dict [{streets_sequence, distance}] for each routes
info_routes = {}
for i, route in enumerate(generated_routes):
    info_routes[i] = {"streets_sequence": route, "distance": distances_travel[i]}



print(f"Maintenance workers : {maintenance_number}\n")

print("Next week routes :\n")
for i, route in enumerate(generated_routes) : 
    print(f"{route} (distance of {round(distances_travel[i],2)})")

Maintenance workers : 2

Next week routes :

[62, 37, 91, 68, 69, 82] (distance of 223.23)
[44, 67, 14, 26] (distance of 98.71)
[5, 49, 16] (distance of 66.49)
[61, 85] (distance of 72.73)
[71, 28, 12, 78, 0] (distance of 162.4)
[20, 38, 70] (distance of 88.38)
[51, 98, 31, 58] (distance of 130.97)
[65, 79, 66] (distance of 134.17)
[53, 9, 42] (distance of 99.9)
[94, 41] (distance of 57.88)
[24, 57] (distance of 43.5)
[22, 59, 90] (distance of 79.73)
[48, 39] (distance of 65.51)
[56, 81, 45, 40, 46] (distance of 127.08)
[89, 54, 6, 60] (distance of 186.17)
[73, 35, 18, 27] (distance of 77.35)
[15, 88] (distance of 36.61)
[96, 30, 1] (distance of 80.0)


### First tree : determine who drives

In [30]:
def first_tree(info, maintenance_number, verbose = True):
    
    #FIRST BRANCH : Take the top 2*maintenance_number workers with the highest driving_index
    info_driving_sort = dict(sorted(info.items(), key=lambda item: item[1]["driving_index"], reverse=True))
    top_workers = list(info_driving_sort.keys())[:2*maintenance_number]
    info_top_workers = {k: info[k] for k in top_workers}

    #SECOND BRANCH : Take the top maintenance_number workers with the highest total_distance of the remaining workers
    info_distance_sort = dict(sorted(info_top_workers.items(), key=lambda item: item[1]["total_distance"], reverse=True))
    top_workers = list(info_distance_sort.keys())[:maintenance_number]
    info_top_workers = {k: info[k] for k in top_workers}

    #Create the driving worker dictionnary
    driving_workers = { k: info[k] for k in info if k not in top_workers}

    if verbose : 
        print("Selected workers for maintenance for next week :")
        for w in info_top_workers:
            print(f"Worker {w} : {info_top_workers[w]}")

        print("\nSelected workers for driving for next week :")
        for w in driving_workers:
            print(f"Worker {w} : {driving_workers[w]}")

    return driving_workers

### Second tree : one line tree where each node is test for a route assignment

In [31]:
def second_tree(driving_workers, info_routes):

    #Sort driving workers by distance (descending)
    driving_workers_sort = dict(sorted(driving_workers.items(), key=lambda item: item[1]["total_distance"], reverse=True))


    #Sort the info_routes to clean by distance (ascending)
    info_routes_sort = dict(sorted(info_routes.items(), key=lambda item: item[1]["distance"], reverse=False))


    assert len(info_routes_sort) == len(driving_workers_sort), "The number of routes to clean and the number of driving workers should be the same"

    #Assign the routes to the driving workers
    worker_match = [-1]*WORKERS_NB
    for i, idx_driver in enumerate(driving_workers_sort):

        for j, idx_route in enumerate(info_routes_sort):
            visited_streets_last_3_weeks = itertools.chain.from_iterable(driving_workers_sort[idx_driver]["visited_streets"].values())
            if not(  set(visited_streets_last_3_weeks) & set(info_routes_sort[idx_route]["streets_sequence"]) ) or j == len(info_routes_sort) - 1: #If the worker has not visited a streets of the route
                assign_route = idx_route #Assign the route to the worker
                break
            
        worker_match[idx_driver] = assign_route

        #delete the route from the info_routes_sort
        del info_routes_sort[assign_route]
        

    return worker_match


### Run the two trees

In [32]:
print("#"*20 + " FIRST TREE " + "#"*20)
driving_workers = first_tree(info, maintenance_number)

print("\n\n" + "#"*20 + " SECOND TREE " + "#"*20)
worker_match = second_tree(driving_workers, info_routes)
print("Worker match : ", worker_match)

#################### FIRST TREE ####################
Selected workers for maintenance for next week :
Worker 7 : {'driving_index': 5.46353356495816, 'total_distance': 2054.642656811898, 'visited_streets': {3: [99, 50, 10, 55], 2: [50, 5, 99, 92, 96], 1: [76, 38, 49, 71, 66, 8]}}
Worker 1 : {'driving_index': 6.4771297856671595, 'total_distance': 1831.2618277594127, 'visited_streets': {3: [18, 44, 81, 47, 7, 3], 2: [74, 39, 57, 98, 17, 21], 1: [83, 15, 75, 14]}}

Selected workers for driving for next week :
Worker 0 : {'driving_index': 5.23294131505816, 'total_distance': 1546.8764676050564, 'visited_streets': {3: [79, 92, 94, 67, 97, 36], 2: [36, 13, 56, 69, 88, 10, 28], 1: [55, 53, 31, 9]}}
Worker 2 : {'driving_index': 5.55810391001755, 'total_distance': 1509.375544132276, 'visited_streets': {2: [30, 68, 49, 91], 1: [43, 81]}}
Worker 5 : {'driving_index': 5.44422418759855, 'total_distance': 1536.1364953336622, 'visited_streets': {3: [98, 40, 6, 56, 57, 68, 49, 62], 2: [97, 58, 72, 46, 1

# 5) Loop on several weeks to check the performance of the our model

Now that we have a proof of concept, let's try to apply our model on several weeks after

In [33]:
def run_one_week(info, distances) :

    #Generate the new routes for the week
    streets_to_clean = generate_streets_to_clean()
    generated_routes, distances_travel = generate_routes(streets_to_clean, distances)

    maintenance_number = WORKERS_NB - len(generated_routes)


    #Put the info in a list of dict [{streets_sequence, distance}] for each routes
    info_routes = {}
    for i, route in enumerate(generated_routes):
        info_routes[i] = {"streets_sequence": route, "distance": distances_travel[i]}

    #First tree
    driving_workers = first_tree(info, maintenance_number, verbose=False)

    #Second tree
    worker_match = second_tree(driving_workers, info_routes)

    #Update the info
    for w in info:
        info[w]["driving_index"] *= GAMMA
        info[w]["visited_streets"] = {k + 1 : v for k,v in info[w]["visited_streets"].items() if k < 3}

        if w in driving_workers:
            route_assigned = worker_match[w]
            info[w]["driving_index"] += GAMMA
            info[w]["total_distance"] += distances_travel[route_assigned]
            info[w]["visited_streets"][1] = generated_routes[route_assigned]



    return info

In [34]:
#########################################
######### SOME PLOT VARIABLES ###########
#########################################
total_distance_evolution = [ [0] for w in info ]
street_0 = [] #Track who visited street 0



#########################################
########## CREATE THE CITY MAP ########## 
#########################################
street_map, distances = generate_streets()



#########################################
###### SIMULATE NB_WEEKS_SIM WEEKS ###### 
#########################################
simulated_weeks = []
for i in tqdm(range(NB_WEEKS_SIM), desc="Simulating weeks..."):

    week_info = {}

    streets_to_clean = generate_streets_to_clean()
    generated_routes, distances_travel = generate_routes(streets_to_clean, distances)
    workers_match, routes_match = generate_match_workers(generated_routes)

    week_info["streets_to_clean"] = streets_to_clean
    week_info["generated_routes"] = generated_routes
    week_info["distances_travel"] = distances_travel
    week_info["workers_match"] = workers_match
    week_info["routes_match"] = routes_match

    simulated_weeks.append(week_info)

    #Update some plot variables
    zero_has_been_visited = False
    for i, w in enumerate(workers_match):
        #Update the total distance evolution
        total_dist = total_distance_evolution[i][-1]
        if w != -1:
            total_dist += week_info["distances_travel"][week_info["workers_match"][i]]
        total_distance_evolution[i].append(total_dist)

        #Update the street 0
        if w != -1 and 0 in week_info["generated_routes"][week_info["workers_match"][i]]:
            street_0.append(i)
            zero_has_been_visited = True
    if not zero_has_been_visited:
        street_0.append(-1)


info = retrieve_info(simulated_weeks)



#########################################
######### RUN THE SIMULATION ############
#########################################
for i in tqdm(range(NB_WEEKS_AFTER), desc="Running simulation..."):
    info = run_one_week(info, distances)

    #Update some plot variables
    zero_has_been_visited = False
    for w in info:
        #Update the total distance evolution
        total_distance_evolution[w].append(info[w]["total_distance"])

        #Update the street 0
        if 1 in info[w]["visited_streets"] and 0 in info[w]["visited_streets"][1] and not zero_has_been_visited :
            street_0.append(w)
            zero_has_been_visited = True
    if not zero_has_been_visited:
        street_0.append(-1)


print("\n\nINFO :")
for w in info:
    print(f"Worker {w} : {info[w]}")

Running simulation...:   0%|          | 0/15 [00:00<?, ?it/s]

Running simulation...: 100%|██████████| 15/15 [00:00<00:00, 699.80it/s]



INFO :
Worker 0 : {'driving_index': 7.220467028342674, 'total_distance': 2946.189921321588, 'visited_streets': {3: [73, 45, 5, 11, 63], 2: [79, 64, 30], 1: [40, 99]}}
Worker 1 : {'driving_index': 6.67937920681946, 'total_distance': 2972.4997188271113, 'visited_streets': {3: [94, 77, 8, 38, 60, 86, 61], 1: [58, 98, 28]}}
Worker 2 : {'driving_index': 6.837123543419054, 'total_distance': 2967.6088881671812, 'visited_streets': {2: [95, 22, 92, 17], 1: [10, 53]}}
Worker 3 : {'driving_index': 6.767808988174838, 'total_distance': 2972.642250265491, 'visited_streets': {2: [36, 67, 56, 47], 1: [72, 60]}}
Worker 4 : {'driving_index': 6.505287821331049, 'total_distance': 3000.7336351225745, 'visited_streets': {2: [99, 89, 16], 1: [68, 36, 69]}}
Worker 5 : {'driving_index': 6.424517439746557, 'total_distance': 3011.0133425081412, 'visited_streets': {2: [34, 2, 32, 23], 1: [39, 6, 80, 44, 16]}}
Worker 6 : {'driving_index': 6.659450905730395, 'total_distance': 2967.054822729954, 'visited_streets':

### Some plots

In [35]:
# Evolution of the total distance traveled by each worker
fig = go.Figure()
for i in range(WORKERS_NB):
    #trace the line 
    fig.add_trace(go.Scatter(x=list(range(NB_WEEKS_SIM + NB_WEEKS_AFTER)), y=total_distance_evolution[i], mode='lines', name=f"Worker {i}"))

#Add a horizontal line for x = NB_WEEKS_SIM
fig.add_shape(type='line',
              x0=NB_WEEKS_SIM, y0=0,
              x1=NB_WEEKS_SIM, y1=max([max(distances) for distances in total_distance_evolution]),
              line=dict(color='Red', width=2, dash='dash'))



fig.update_layout(title='Evolution of the total distance traveled by each worker', xaxis_title='Weeks', yaxis_title='Total distance traveled')
fig.show()


In [36]:
#### Who visited street 0 ?
print("DURING THE SIMULATION, WHO VISITED STREET 0 ?")
string_print = ""
for i, w in enumerate(street_0[:NB_WEEKS_SIM]):
    if i != 0 :
        string_print += " -> "
    
    if w != -1:
        string_print += f"{w}"
    else:
        string_print += "//"

print(string_print)

print("\n\nAFTER THE SIMULATION, WHO VISITED STREET 0 ?")
string_print = ""
for i, w in enumerate(street_0[NB_WEEKS_SIM:]):
    if i != 0 :
        string_print += " -> "
    
    if w != -1:
        string_print += f"{w}"
    else:
        string_print += "//"
print(string_print)


DURING THE SIMULATION, WHO VISITED STREET 0 ?
// -> // -> 18 -> 11 -> 4 -> 6 -> // -> // -> 5 -> // -> 12 -> 12 -> // -> // -> //


AFTER THE SIMULATION, WHO VISITED STREET 0 ?
4 -> 4 -> // -> // -> // -> 6 -> 19 -> 13 -> // -> 3 -> 14 -> 4 -> // -> // -> 17
